In [206]:
import pandas as pd 
import numpy as np 
from numpy import array 
from sklearn.preprocessing import OneHotEncoder
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
from pymorphy2 import MorphAnalyzer
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import BayesianRidge 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 8.2 MB 68.9 MB/s 


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [243]:
data = pd.read_csv('data.csv') 

In [244]:
y = data.bugs
X = data.drop(columns = ['bugs'])

In [245]:
unique_repository_name = []
for number in X['repository_name']:
    if number not in unique_repository_name:
        unique_repository_name.append(number)
unique_repository_name

['mlm', 'standard', 'agent', 'conductor', 'dockers', 'sensor']

In [246]:
unique_commit_author = []
for num in X['commit_author']:
    if num not in unique_commit_author:
        unique_commit_author.append(num)
unique_commit_author

['Victor',
 'Peggy',
 'Wendy',
 'Trudy',
 'Mallory',
 'Eve',
 'Dabe',
 'Carol',
 'Bob',
 'Alice']

In [247]:
author = array(X.commit_author).reshape(-1, 1)

In [248]:
onehotencoder = OneHotEncoder()
data_new = onehotencoder.fit_transform(author)
auth = pd.DataFrame(data_new.toarray(), columns=onehotencoder.categories_)

In [249]:
for i in unique_commit_author:
  X[i] = auth[i]

In [250]:
X = X.drop(columns = ['commit_author'])

In [251]:
rep_name = array(X.repository_name).reshape(-1, 1)

In [252]:
onehotencoder = OneHotEncoder()
rep_name_data = onehotencoder.fit_transform(rep_name)
repo_name = pd.DataFrame(rep_name_data.toarray(), columns=onehotencoder.categories_)

In [253]:
for i in unique_repository_name:
  X[i] = repo_name[i]
X = X.drop(columns = ['repository_name'])

In [254]:
commit_date = X['commit_date']
month_ = pd.DatetimeIndex(commit_date).month
day_ = pd.DatetimeIndex(commit_date).day
hour_ = pd.DatetimeIndex(commit_date).hour

In [255]:
X['commit_month'] = month_
X['commit_day'] = day_
X['commit_hour'] = hour_
X = X.drop(columns = ['commit_date'])

In [256]:
commit_message = X['commit_message']

In [257]:
stop_words = stopwords.words("russian") + stopwords.words('english')
patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
morph = MorphAnalyzer()

In [258]:
filtered_elem = []
message_ = []
for elem in commit_message:
  elem = re.sub(patterns, ' ', elem)
  for token in elem.split():
    if token not in stop_words:
      token = token.strip()
      token = morph.normal_forms(token)[0]
      filtered_elem.append(token)

  message_.append(filtered_elem)
  filtered_elem = []


In [259]:
X['commit_mess'] = message_
X = X.drop(columns = ['commit_message'])

In [260]:
def conect(array):
    string = ""
    for i in range(len(array)):
        string += str(array[i])+' '
    return string.rstrip()

In [261]:
X['commit_mess'] = X['commit_mess'].apply(conect)

In [262]:
X = X.drop(columns = ['commit_hash'])

In [263]:
vectorizer = TfidfVectorizer(min_df=5, max_df=0.9)

In [264]:
message_vect = vectorizer.fit_transform(X['commit_mess'])

In [265]:
df_message = pd.DataFrame(message_vect.toarray())

In [267]:
X = X.drop(columns = ['commit_mess'])

In [268]:
for el in df_message:
  X[el] = df_message[el]

In [291]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [292]:
def print_metrics(y_preds, y):
    print(f'R^2: {r2_score(y_preds, y)}')
    print(f'MSE: {mean_squared_error(y_preds, y)}')
    print(f'Accuracy: {accuracy_score(y_preds, y)}')

In [293]:
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, y_train)

print_metrics(knn.predict(X_test), y_test)

R^2: -0.8106235565819861
MSE: 5.359375
Accuracy: 0.296875


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [294]:
bayes = GaussianNB()
bayes.fit(X_train, y_train)

print_metrics(bayes.predict(X_test), y_test)

R^2: -0.22420266371576503
MSE: 7.5625
Accuracy: 0.15625


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


Итог: получилось не очень :-<